In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pickle
import numpy as np

In [ ]:

def load_cifar_batch(pathname,i):
    with open(pathname,'rb') as f:
        datadict = pickle.load(f,encoding='bytes') 
        #print(datadict)
        batch_labels = datadict[b'labels']
        batch_data = datadict[b'data']
        batch_data =batch_data.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        if i==1:
            #print(batch_data.shape)
            plt.imshow(batch_data[0].astype('uint8'))
        batch_labels = np.array(batch_labels)
    return batch_data,batch_labels      
def load_cifar10():
    x_data = []
    y_data = []
    for i in range(1,6):
        path=os.path.join('/kaggle/input/cifar10/cifar-10-batches-py','data_batch_{}'.format(i))
        batch_x,batch_y = load_cifar_batch(path,i)
        x_data.append(batch_x)
        y_data.append(batch_y)
    Xtr= np.concatenate(x_data) 
    Ytr= np.concatenate(y_data)
    Xte,Yte = load_cifar_batch(os.path.join('/kaggle/input/cifar10/cifar-10-batches-py','test_batch',),i)
    return Xtr,Ytr,Xte,Yte

In [ ]:
Xtrain,Ytrain,Xtest,Ytest=load_cifar10()
print(Xtrain.shape)
print(Ytrain.shape)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(Ytrain == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(Xtrain[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()
#print(Xte.shape)
#print(Yte.shape)

In [ ]:
Xtrain_reshaped = Xtrain.reshape(-1,32*32*3)
Xtest_reshaped = Xtest.reshape(-1,32*32*3)
num_train = 49000
num_val = 1000
num_test=1000
num_dev = 500

mask= range(num_train,num_train+num_val)
X_val = Xtrain_reshaped[mask]    
Y_val=Ytrain[mask]

mask = range(num_train)
X_train = Xtrain_reshaped[mask]
Y_train = Ytrain[mask]

mask = np.random.choice(num_train, num_dev, replace=False)
X_dev = X_train[mask]
Y_dev = Y_train[mask]

mask = range(num_test)
X_test = Xtest_reshaped[mask]
y_test = Ytest[mask]


In [ ]:
import matplotlib.pyplot as plt
def process_data(X_train,X_test,X_val,X_dev):
    mean_train = np.mean(Xtrain_reshaped,axis=0)
    mean_test = np.mean(Xtest_reshaped,axis=0)
    print(mean_train[:10]) # print a few of the elements
    plt.figure(figsize=(4,4))
    plt.imshow(mean_train.reshape((32,32,3)).astype('uint8')) # visualize the mean image
    plt.show()
    X_train-=mean_train.astype('uint8')
    X_val-=mean_train.astype('uint8')
    X_test-=mean_train.astype('uint8')
    X_dev-=mean_train.astype('uint8')
    #print(X_train.dtype)
    #X_train=X_train//255
    #X_val//=255
    #X_test//=255
    #X_dev//=255
    #X_train/=255
    #X_test/=255
    return X_train,X_val,X_test,X_dev

In [ ]:
X_train,X_val,X_test,X_dev = process_data(X_train,X_test,X_val,X_dev)
X_train = np.hstack([X_train,np.ones((X_train.shape[0],1))])
X_val = np.hstack([X_val,np.ones((X_val.shape[0],1))])
X_dev = np.hstack([X_dev,np.ones((X_dev.shape[0],1))])
X_test = np.hstack([X_test,np.ones((X_test.shape[0],1))])

In [ ]:
def vectorized_svm_loss(X,W,y,reg):
    delta=1
    dW = np.zeros(W.shape)
    num_train = X.shape[0]
    num_class = W.shape[1]
    out = np.dot(X,W)
    margins = out - out[np.arange(num_train),y].T[:,None]+delta
    margins[margins<0]=0
    margins[np.arange(num_train),y] = 0
    loss=margins
    loss = np.sum(loss)
    margins[margins>0]=1
    rowsum = np.sum(margins,axis=1)
    margins[np.arange(num_train),y] = -rowsum
    dW = np.dot(X.T,margins)
    dW/=num_train
    loss/=num_train
    loss+=0.5*reg*np.sum(W*W)
    dW+=reg*W
    return dW,loss

In [ ]:
# Once you've implemented the gradient, recompute it with the code below
# and gradient check it with the function we provided for you
import random
# Compute the loss and its gradient at W.
grad,loss = vectorized_svm_loss(X_dev,W, Y_dev, 0.0)

# Numerically compute the gradient along several randomly chosen dimensions, and
# compare them with your analytically computed gradient. The numbers should match
# almost exactly along all dimensions.
#def grad_check_sparse():
def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5):
  """
  sample a few random elements and only return numerical
  in this dimensions.
  """

  for i in range(num_checks):
    ix = tuple([random.randrange(m) for m in x.shape])

    oldval = x[ix]
    x[ix] = oldval + h # increment by h
    fxph = f(x) # evaluate f(x + h)
    x[ix] = oldval - h # increment by h
    fxmh = f(x) # evaluate f(x - h)
    x[ix] = oldval # reset

    grad_numerical = (fxph - fxmh) / (2 * h)
    grad_analytic = analytic_grad[ix]
    rel_error = abs(grad_numerical - grad_analytic) / (abs(grad_numerical) + abs(grad_analytic))
    print('numerical: %f analytic: %f, relative error: %e' % (grad_numerical, grad_analytic, rel_error))
    
f = lambda w: vectorized_svm_loss(X_dev,w, Y_dev, 0.0)[1]
grad_numerical = grad_check_sparse(f, W, grad)

In [ ]:
import time
num_class = 10
print(X_train.shape)
W = np.random.randn(X_train.shape[1],num_class)*0.0001
tic = time.time()
reg = 1e-5
xbatch = X_train[0:100]
ybatch = Y_train[0:100]
grad,loss = vectorized_svm_loss(xbatch,W,ybatch,reg)
toc = time.time()
print(toc-tic)
print(grad,loss)

In [ ]:
def predict(X_train,X_val,W):
    train_pred = np.argmax(np.dot(X_train,W),axis=1)  
    val_pred = np.argmax(np.dot(X_val,W),axis=1)  
    return train_pred,val_pred

In [ ]:
from sklearn.utils import shuffle
def train(X_train,Y_train,X_val,Y_val,W,eta,reg):
    num_train = X_train.shape[0]
    num_val = X_val.shape[0]
    trainloss_array = []
    valloss_array = []
    #reg = 1e+03
    
    tot_loss=0
    batch_size=100
    for i in range(num_train//batch_size):
        batch = X_train[i*batch_size:(i+1)*batch_size]
        label = Y_train[i*batch_size:(i+1)*batch_size]
        grad,loss = vectorized_svm_loss(batch,W,label,reg)
        #print(loss)
        #print(grad)
        W-=eta*grad
        tot_loss+=loss
    train_loss=tot_loss/(num_train//batch_size)
    #print(train_loss)
    tot_loss=0    
    for i in range(num_val//batch_size):
        val_batch = X_val[i*batch_size:(i+1)*batch_size]
        val_label = Y_val[i*batch_size:(i+1)*batch_size]
        grad,valloss = vectorized_svm_loss(val_batch,W,val_label,reg)    
        tot_loss+=valloss
    val_loss = tot_loss/(num_val//batch_size)
    return train_loss,val_loss,W

In [ ]:
train_loss = []
val_loss = []
etas=[]
epochs = 80
eta=0.0000001
num = 30

learning_rates = 10 ** np.random.uniform(-7, -6, size=num)
regs = 10 ** np.random.uniform(3.5, 4.5, size=num)
t_acc = []
v_acc = []
for lr,reg in zip(learning_rates,regs):
    W = np.random.randn(X_train.shape[1],num_class)*0.001
    for epoch in range(epochs):
        X_train,Y_train = shuffle(X_train,Y_train)
        trloss,valloss,W = train(X_train,Y_train,X_val,Y_val,W,lr,reg)
    trpred,vlpred = predict(X_train,X_val,W)    
    train_acc = np.mean(trpred==Y_train)
    val_acc = np.mean(vlpred==Y_val)
    t_acc.append(train_acc)
    v_acc.append(val_acc)
    train_loss.append(trloss)    
    val_loss.append(valloss)
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss)
plt.show()
plt.figure()
plt.plot(val_loss)
plt.figure()
plt.plot(v_acc)
plt.figure()
plt.plot(t_acc)
print(val_loss)

In [ ]:
#pred_index = np.argmax(np.dot(X_test[0],W))
#correct_label = y_test[0]
positive=0

positive=0
for i in range(X_test.shape[0]):
    correct_label = y_test[i]
    pred_index = np.argmax(np.dot(X_test[i],W))
    if(pred_index==correct_label):
        positive+=1
test_acc = positive/X_test.shape[0]
print('train_acc: {}'.format(train_acc*100),'test_acc: {}'.format(test_acc*100),'val_acc: {}'.format(val_acc*100))

In [ ]:
print(learning_rates[17],regs[17])
lr,reg = learning_rates[17],regs[17]
W = np.random.randn(X_train.shape[1],num_class)*0.001
for epoch in range(epochs):
    X_train,Y_train = shuffle(X_train,Y_train)
    trloss,valloss,W = train(X_train,Y_train,X_val,Y_val,W,lr,reg)
#    trpred,vlpred = predict(X_train,X_val,W)    
#    train_acc = np.mean(trpred==Y_train)
#    val_acc = np.mean(vlpred==Y_val)
#    t_acc.append(train_acc)
#    v_acc.append(val_acc)
print(trloss)
print(valloss)

In [ ]:
w = W[:-1,:]
w = w.reshape(32,32,3,10)
np.savez('weights.npz',name1=W,name2=w)
minw,maxw = np.ndarray.min(w),np.ndarray.max(w)
print(minw,maxw)
for i in range(10):
    plt.subplot(2, 5, i + 1)
    tempw =255.0*(w[:,:,:,i].squeeze()-minw)/(maxw-minw)
    #print(tempw)
    plt.imshow(tempw.astype('uint8'))
    

In [ ]:
trpred,vlpred = predict(X_train,X_val,W)    
train_acc = np.mean(trpred==Y_train)
val_acc = np.mean(vlpred==Y_val)
print("val_acc:{}".format(val_acc),"train_acc:{}".format(train_acc))